In [2]:
# Checks if two points are within a certain range of one another
def inRange(p1, p2, tol): # p1 newpoint p2 currentpoint
    yscale = 3

    xbool = p1[0]-tol <= p2[0] <= p1[0]+tol
    ybool = p1[1]-tol*yscale <= p2[1] <= p1[1]+tol*yscale

    return xbool and ybool

def IOU(base, unk):
        x1, y1 = base[0]-base[2]/2, base[1]-base[3]/2
        x2, y2 = base[0]+base[2]/2, base[1]+base[3]/2
        x3, y3 = unk[0]-unk[2]/2, unk[1]-unk[3]/2
        x4, y4 = unk[0]+unk[2]/2, unk[1]+unk[3]/2

        # Calculate area of intersection
        x_overlap = max(0, min(x2,x4) - max(x1,x3))
        y_overlap = max(0, min(y2,y4) - max(y1,y3))
        intersection_area = x_overlap * y_overlap

        # Calculate area of union
        box1_area = base[2] * base[3]
        box2_area = unk[2] * unk[3]
        union_area = box1_area + box2_area - intersection_area

        # Calculate IoU
        iou = intersection_area / union_area
        
        return iou

def scale_image(image, scale):
    x = image.shape[1]
    y = image.shape[0]
    image = cv2.resize(image, ( int(x*scale), int(y*scale) ))
    
    return image


# Squares images based on length does not interact with one dimension
def square_photos(images, resize=False):
    
    newImgs = []
    
    for label in images:
        shape = images[label].shape

        # If long then reduce both sides equally until research desired size
        if shape[0] < shape[1]: # Long
            r = int( (shape[1] - shape[0]) / 2 ) 

            images[label] = images[label][:,r : shape[0] + r] 
            
        # If Tall then reduce both sides equally until research desired size
        if shape[0] > shape[1]: # Tall 
            r = int( (shape[0] - shape[1]) / 2 )

            images[label] = images[label][r : shape[1] + r]
        
        # Resize all images to the same size if required
        if type(resize) == int:
            images[label] = cv2.resize(images[label], (resize, resize) )
    
    return images

    
# Scales normalized data to the image
def scale_detections(det, img):
    
    # Remakes array with resized objects 
    det[1::2] = [ int(x * img.shape[1]) for x in det[1::2] ] # X-Values
    det[2::2] = [ int(y * img.shape[0]) for y in det[2::2] ] # Y-Values
    
    return det

# Determines the most significate object in an image
def mostSigObject(detects):
    
    zero = 0.01 # Machine zero
    
    unique = []
    
    for i, det in enumerate(detects):
        
        if det == None or det.shape[0] == 0:
            flag = False

            # If no image has the same classes then return false
            # print("No Valid Detections")
            return flag, detects

        det = np.array(det)
        
        # Delete Objects which border the image
        rangeMask = (det[:,1] - (det[:,3]/2) >= zero) \
                    & (1 - det[:,2] - (det[:,4]/2) >= zero) \
                    & (1 - det[:,1] - (det[:,3]/2) >= zero) \
                    & (det[:,2] - (det[:,4]/2) >= zero)
        
        # Overlays mask to remove objects
        detects[i] = det[rangeMask]
        
        # Find unique classes in image
        uniqueClasses = np.unique(np.array(detects[i][:,0]))
        unique.extend([*uniqueClasses])
        
    unique = np.unique(unique, return_counts=True)# 
    # unique[0] = unique[0][::-1] # If decided to reverse the classes
    
    # The index of which value to use
    index = 0
    
    if len(unique[0]) == 0:
        flag = False
        
        # print("Objects detected but all Invalid")
        return flag, detects
    
    # Loops finds the first class with the same number of objects as images
    for i in range(len(unique[0])): ### Maybe instead of the first calculate who has higher confidence 
        index = i
        
        if unique[1][i] == 3:
            break
        if i == len(unique[0])-1:
            flag = False
            
            # If no image has the same classes then return false
            # print("All images do not have same class")
            return flag, detects
        
    # Object class which is in all images
    comparedClass = unique[0][index]
    # print("comparedClass: ", comparedClass)
    
    output = []
    
    # Create output array of single objects assumed to be same object
    for i, det in enumerate(detects):
        classMask = (det[:,0] == comparedClass)
        det = det[classMask]
        
        # Sorts objects by size 
        detects[i] = np.array(sorted(det, key=lambda det : det[-1]*det[-2], reverse=True))
        
        # Grabs the largest object from each image
        output.append(detects[i][0])
    
    # print("Largest Objects selected \n", np.array(output))
    
    tol = 0.15 # Tolerance range an object can be within to be the same
    
    # Check if all objects are within range
    # print(output)
    
    iou = [IOU(output[0][1:], output[1][1:]), 
               IOU(output[0][1:], output[2][1:]),
               IOU(output[1][1:], output[2][1:])]
    
    minIOU = np.mean(iou)
    tol = 0.2
    
    flag = inRange(output[0][1:3], output[1][1:3], tol) \
    & inRange(output[0][1:3], output[2][1:3], tol) \
    & inRange(output[1][1:3], output[2][1:3], tol)
    
    if minIOU > tol and flag:
        flag = True
        
    else:
        # print("Objects are not within Range")
        flag = False
    
    output = np.array(output) 
    
    # Return if objects are valid and output those objects
    return flag, output


# Calculates relationship of all images and records relationship
def calibration(images, model, minimalImg, view_relationship=True):
    
    imagesUsed = ["RGB", "Thermal", "Left"]
    
    # Square incomming images to preserve ratio between all images
    images = square_photos(images)
    
    multiImageDetections = []
    
    # Goes through each image to gather detections
    for label in imagesUsed:
        
        det = model.predict(source=images[label], classes=[0,2], verbose=False, device=0)
        det = torch.cat((det[0].boxes.cls.view(-1,1), det[0].boxes.xywhn), dim=1)
        
        multiImageDetections.append(det.cpu())
        
    # Returns if objects are identical or not
    flag, multiImageDetections = mostSigObject(multiImageDetections)
    
    
    # If not identical error out
    if not flag:
        return None, None
    
    multiImageDetections = {"RGB": multiImageDetections[0], "Thermal": multiImageDetections[1], "Left": multiImageDetections[2]}
    
    
    # Selects most important feature of object, length or height
    item = {2.0 : -2, 0.0 : -1}
    item = item[multiImageDetections[minimalImg][0]]
    
    
    # Finds smallest object, assumes this camera holds most infomation
    min_size = multiImageDetections[minimalImg][item]
    # min_idx = np.argmin(multiImageDetections[:,item])
    # min_size = images[minimalImg].shape[0]
    min_idx = minimalImg
    
    # Scale detections of minimal image since it will not change
    multiImageDetections[minimalImg] = scale_detections( multiImageDetections[minimalImg], images[minimalImg] )
    
    scales = []
    offset = []
    
    # Go through each photo and calculate the relationship of each photo in relation to the smallest
    for label in imagesUsed:
        
        # If smallest image than set relationship to None
        if label == minimalImg:
            
            # If the minimal image is encountered then relationship is change nothing
            scales.append(1)
            offset.append([0,0,0,0])
            continue
        
        # Calculate scale based on difference of item size
        # print(multiImageDetections[label])
        scale = (min_size * images[minimalImg].shape[0]) / (multiImageDetections[label][item] * images[label].shape[0])
        
        # Resize images based on scale
        images[label] = scale_image(images[label].copy(), scale)
        
        # After resize scale detections
        singleImageDetection = scale_detections(multiImageDetections[label], images[label])
        
        # What is the difference in resolution 
        resolutionDiff_x = images[minimalImg].shape[1] - images[label].shape[1] # All space for x
        resolutionDiff_y = images[minimalImg].shape[0] - images[label].shape[0] # All space for y
        
        # Compare how offset an object is compared to minimal image
        offset_x = multiImageDetections[minimalImg][1] - singleImageDetection[1]
        offset_y = multiImageDetections[minimalImg][2] - singleImageDetection[2]
        
        # Use left and top to move objects into place and bottom and right to simply make images same ints required
        top = int(offset_y)
        bottom = int(resolutionDiff_y - offset_y)
        left = int(offset_x)
        right = int(resolutionDiff_x - offset_x)
        
        # Create offset and scale values, this is the relationship between the images
        offset.append([top, bottom, left, right])
        scales.append(scale)
    
        # View in image how object must be translated to match minimal image
        if view_relationship:
            
            relationshipImg = images[label].copy()
            x = int(singleImageDetection[1])
            y = int(singleImageDetection[2])
            
            relationshipImg = cv2.line(relationshipImg, (x, y), (x + left, y), (0,255,0), 2)
            relationshipImg = cv2.line(relationshipImg, (x, y), (x, y + top), (0,0,255), 2)
            
            cv2.imshow("cali_"+label, relationshipImg)
            cv2.waitKey(1)
                
    
    # minImageSize = images[minimalImg].shape[0]
    
    return scales, offset


### Still have to figure out negative tops, and rights
### Getting good idea about fixing it, fuck the bottom and right, focus on left and top, then just go the nessassary amount
# Adds relationship infomation to images to make same as minimal image
def modify_images(images, scales, offsets, minImageSize):
    imagesUsed = ["RGB", "Thermal", "Depth"]
    
    # Alter images to match minimal image
    for label in imagesUsed:
        
        if scales[label] == 1:
            images[label] = square_photos({label: images[label]})[label]
            continue
        
        # Scale Images
        images[label] = scale_image(images[label], scales[label])
        
        top, bottom, left, right = offset[label]
        
        # Used to orient everything from square image
        p = int( (images[label].shape[1] - images[label].shape[0]) / 2 ) ### Not sure what to name this variable
        
        # Supports negative and positive left values
        l = p - left 

        # Removes unnessassary pixel values 
        images[label] = images[label][:, l: l+minImageSize]
        
        # If there is still space requied, then fill it with black space
        right = int(minImageSize - images[label].shape[1])
        
        if top < 0:
            images[label] = images[label][-top:minImageSize-top]
            top = 0
        else:
            images[label] = images[label][:minImageSize-top]
            
            
        bottom = int(minImageSize - images[label].shape[0] - top)
        
        # Add borders to image to fit ### abs is just error preventation since I havent done negatives for some of them yet
        images[label] = cv2.copyMakeBorder(images[label], abs(top), abs(bottom), 0, abs(right), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        
    return images


# Mask the images to create the SII
def apply_mask(images, masks, mainImg):
    
    # Copy image as to not affect main image
    superImposedImage = images[mainImg].copy()
    
    masterMask = []
    
    # Add all mask together
    if masks != None:
        for m in masks:
            
            m = np.array(m)

            # Scale Mask to Images
            m = cv2.resize(m, superImposedImage.shape[:2][::-1])

            if len(masterMask) == 0:
                masterMask = m
            else:
                masterMask += m
    
    else:
        return superImposedImage
    
    # Invert masterMask because this will delete objects 
    masterMask = 1 - masterMask
    
    # Change to bool to effect images
    masterMask = masterMask.astype(bool)
    
    # Set pixels not related to objects to zero
    thermalMasked = images["Thermal"].copy() # Images must be copied for or else its gonna mess with original images
    stereoMasked = images["Color_Depth"].copy() 
    # stereoMasked = cv2.cvtColor(stereoMasked, cv2.COLOR_GRAY2BGR)
    
    
    if thermalMasked.shape != masterMask.shape:
        thermalMasked = cv2.resize(thermalMasked, masterMask.shape[:2][::-1]) 
        stereoMasked = cv2.resize(stereoMasked, masterMask.shape[:2][::-1]) 
        superImposedImage = cv2.resize(superImposedImage, masterMask.shape[:3][::-1]) 
    
    # Set pixels not related to objects to zero
    thermalMasked[masterMask] = 0
    stereoMasked[masterMask] = 0
    
    
    # Display the images to see if mask is working correctly
    cv2.imshow("thermalMasked", thermalMasked)
    cv2.imshow("stereoMasked", stereoMasked)
    cv2.waitKey(1)
    
    # Mask non-required pixels 
    mask = stereoMasked.astype(bool)
    
    # Add thermal pixel infomation
    superImposedImage[mask] = cv2.addWeighted(superImposedImage, 0.70, stereoMasked, 0.30, 0)[mask]
    
    # Mask non-required pixels 
    mask = thermalMasked.astype(bool)
    
    # Add thermal pixel infomation
    superImposedImage[mask] = cv2.addWeighted(superImposedImage, 0.65, thermalMasked, 0.35, 0)[mask]
    
    # Finally return image 
    return superImposedImage
    
    
# A simple overlay function for showing superimposition without adding relationship
def superimpose_images(images, resize=512, addColor=False):
    
    images = square_photos(images, resize)
    
    if addColor:
        images["Depth"] = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
        images["Depth"] = cv2.equalizeHist(images["Depth"])
        images["Depth"] = cv2.applyColorMap(images["Depth"], cv2.COLORMAP_JET)
    
    superImposedImage = cv2.addWeighted(images["RGB"], 0.70, images["Depth"], 0.30, 0)
    superImposedImage = cv2.addWeighted(superImposedImage, 0.65, images["Thermal"], 0.35, 0)
    
    return superImposedImage

def filter_offsets_scales(TenScales, TenOffsets, scale_filter=np.median, offset_filter=np.median):
    
    filteredScales = [ scale_filter([i[0] for i in TenScales]), scale_filter([i[1] for i in TenScales]),  scale_filter([i[2] for i in TenScales]) ]
    
    filteredOffset = [[],[],[]]
    filteredOffset[0] = [int(offset_filter([i[0][0] for i in TenOffsets])), int(offset_filter([i[0][1] for i in TenOffsets])), int(offset_filter([i[0][2] for i in TenOffsets])), int(offset_filter([i[0][3] for i in TenOffsets]))]
    filteredOffset[1] = [int(offset_filter([i[1][0] for i in TenOffsets])), int(offset_filter([i[1][1] for i in TenOffsets])), int(offset_filter([i[1][2] for i in TenOffsets])), int(offset_filter([i[1][3] for i in TenOffsets]))]
    filteredOffset[2] = [int(offset_filter([i[2][0] for i in TenOffsets])), int(offset_filter([i[2][1] for i in TenOffsets])), int(offset_filter([i[2][2] for i in TenOffsets])), int(offset_filter([i[2][3] for i in TenOffsets]))]
    
    return filteredScales, filteredOffset